In [28]:
import tensorflow as tf
import numpy as np
from utils.neuralnets.ResNet110 import ResNet110
import time

In [29]:
##load data and standardize
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = (x_train - np.mean(x_train,axis=0))/np.std(x_train,axis=0)
x_test = (x_test - np.mean(x_test,axis=0))/np.std(x_test,axis=0)
##train validation split, 45000 for training and 5000 for validation
np.random.seed(42)
mask_val = np.random.choice(50000,5000,replace=False)
mask_train = np.array([i for i in range(50000) if i not in mask_val])
x_val, y_val = x_train[mask_val], y_train[mask_val]
x_train, y_train = x_train[mask_train], y_train[mask_train]

In [39]:
##data augmentation
##augmented with horizontal flip and random erasing
def randomErasing(x, cut_size=16):
    x = np.copy(x)
    fill = x.mean()

    h, w, _ = x.shape
    top = np.random.randint(0 - cut_size // 2, h - cut_size)
    left = np.random.randint(0 - cut_size // 2, w - cut_size)
    bottom = top + cut_size
    right = left + cut_size
    if top < 0:
        top = 0
    if left < 0:
        left = 0
    x[top:bottom, left:right, :].fill(fill)
    return x
batch_size = 128
img_datagen = ImageDataGenerator(horizontal_flip=True,preprocessing_function = randomErasing)
train_data = img_datagen.flow(x_train,y_train,batch_size=batch_size)
validation_data = datagen_for_test.flow(x_val, y_val, batch_size=batch_size)

In [40]:
##create a ResNet110 model
input_shape = x_train.shape[1:]
num_class = 10
model = ResNet110(input_shape=input_shape,num_class=num_class)

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1,nesterov=True)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()


epochs = 200
history = [[],[],[]]
for epoch in range(epochs):
    print("Epoch %d/%d" % (epoch+1,epochs))
    if (epoch==100) | (epoch==150):
        optimizer.learning_rate = optimizer.learning_rate/10
    
    start_time = time.time()
    step = 0
    for x_batch_train, y_batch_train in train_data:
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        train_acc_metric.update_state(y_batch_train, logits)
        if step % 25 == 0:
            print(
                "Training loss at step %d: %.4f"
                % (step, float(loss_value))
            )
        step += 1
        if step > len(x_train)/batch_size:
            break

    history[0].append(loss_value)
    train_acc = train_acc_metric.result()
    train_acc_metric.reset_states()
    
    step = 0
    for x_batch_val, y_batch_val in validation_data:
        val_logits = model(x_batch_val, training=False)
        val_acc_metric.update_state(y_batch_val, val_logits)
        step += 1
        if step > len(x_val)/batch_size:
            break
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    history[1].append(train_acc)
    history[2].append(val_acc)
    print("Training accuracy: %.4f" % (float(train_acc),)
          ,"Validation accuracy: %.4f" % (float(val_acc),),"Time taken: %.2fs" % (time.time() - start_time))


Epoch 1/200
Training loss at step 0: 17.1206
Training loss at step 25: 6.1403
Training loss at step 50: 2.6946
Training loss at step 75: 2.7706
Training loss at step 100: 2.5718
Training loss at step 125: 2.2261
Training loss at step 150: 2.2935
Training loss at step 175: 2.3775
Training loss at step 200: 2.1292
Training loss at step 225: 2.1434
Training loss at step 250: 2.1994
Training loss at step 275: 2.0656
Training loss at step 300: 2.0139
Training loss at step 325: 2.0517
Training loss at step 350: 2.0764
Training accuracy: 0.1773 Validation accuracy: 0.2278 Time taken: 1390.33s
Epoch 2/200
Training loss at step 0: 1.9938
Training loss at step 25: 2.0927
Training loss at step 50: 1.9621
Training loss at step 75: 1.9845
Training loss at step 100: 1.8682
Training loss at step 125: 1.8642
Training loss at step 150: 1.9370
Training loss at step 175: 1.8850
Training loss at step 200: 1.8585
Training loss at step 225: 1.7437
Training loss at step 250: 1.8139
Training loss at step 275: